## Gradient Boosting Classification

Gradient boosting is an alternative form of Adaboost. Many consider gradient boosting to be a better performer than adaboost.Some differences between the two algorithms is that gradient boosting uses optimization for weight the estimators. Like adaboost, gradient boosting can be used for most algorithms but is commonly associated with decision trees.

In addition , gradient boosting requires several additional hyper parameters such as max depth and subsample. Max depth has to do with the number of nodes in a tree. The higher the number the purer the classification become. The downside to this is the risk of overfitting.


Subsampling has to do with the proportion of the sample that is used for each estimator. This can range form a decimal value up until the whole number 1. If the value is set to 1 it becomes stochastic gradient boosting.

This post is focused on classification. To do this, we will use the cancer dataset from the pydataset library. Our goal will be to predict the status of patients(alive or dead)using the available independent variables. The steps we will use as follows.

- Data Preparation
- Baseline decision tree model
- Hyperparameter tuning 

#### Importing neccesary packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from sklearn import tree
from pydataset import data
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

The data preperation is simple in this situation. All we need to do is load are dataset, dropping missing values, and create our X dataset and y dataset. All this happens in the code below

In [3]:
df = data('cancer').dropna()
X = df[['time','sex','ph.karno','pat.karno', 'meal.cal', 'wt.loss']]
y = df['status']

We will create now our baseline model

#### Baseline Model

The purpose of the baseline model is to have something to compare our gradient boosting model to. The strength of a model is always relative to some other model, so we need to make at least two, so we can say one is better than other.

The criteria in this situation is accuracy.Therefore, we will make a decision tree model. But we will manipulate the max depth of the tree to create 9 different baseline models. The best accuracy model will be the baseline model.

To achieve this we need to use a for loop to make python make several decision trees. We also need to set the parameters for the cross validation by calling KFold(). Once this done, we print the result for the 9 trees.

In [4]:
crossvalidation = KFold(n_splits=10,shuffle=True,random_state=1)
for depth in range(1,10):
    tree_classifier = tree.DecisionTreeClassifier(max_depth = depth, random_state = 1)
    if tree_classifier.fit(X,y).tree_.max_depth<depth:
        break
    score = np.mean(cross_val_score(tree_classifier,X,y,scoring = 'accuracy',cv = crossvalidation,n_jobs = 1))
    print(depth,score)

1 0.71875
2 0.6477941176470589
3 0.6768382352941177
4 0.6698529411764707
5 0.6584558823529412
6 0.6525735294117647
7 0.6283088235294118
8 0.6573529411764706
9 0.6577205882352941


It appears that when the max depth is limited to 1 that we get the best accuracy at almost  72%. This will be our baseline for comparison. We will tune the parameters for the gradient boosting algorithm

#### Hyperparameter Tuning

There are several hyperparameters we need to tune. The ones we will tune as fallows
- number of estimators 
- learning rate 
- subsample
- max_depth

First we will create an instance of the gradient boosting classifier.Second we will create our grid for the search. It is inside this grid that we set several values for each hyperparameter. Then we call GridSearchCV and place the instance of the gradient boosting classifier, the grid, the cross validation values from mad earlier, and n_jobs all together in one place. Below is the code for this


In [5]:
parameter = {
    'n_estimators':[500,1000,2000],
    'learning_rate':[.001,0.01,.1],
    'max_depth':[1,3,5],
    'subsample':[.5,.75,1],
    'random_state':[1]
}

classifier = GradientBoostingClassifier()

In [6]:
classifiercv=GridSearchCV(classifier,param_grid=parameter,cv=crossvalidation,scoring='accuracy')
classifiercv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=10, random_state=1, shuffle=True),
             estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [1, 3, 5],
                         'n_estimators': [500, 1000, 2000], 'random_state': [1],
                         'subsample': [0.5, 0.75, 1]},
             scoring='accuracy')

In [7]:
classifiercv.best_params_

{'learning_rate': 0.001,
 'max_depth': 5,
 'n_estimators': 1000,
 'random_state': 1,
 'subsample': 0.75}

In [8]:
classifiercv.best_score_

0.7301470588235295

You can see that the best hyper parameters are for yourself. In addition, we see that when these parameters were set we got an accuracy of 74%. This is superior to our baseline model.We will now see if we can replicate these numbers when we use them for our Gradient Boosting model

#### Gradient Boosting Model

Below is the code and results for the model with the predetermined hyperparameter values.

In [9]:
ada2 = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.001, subsample=0.75, max_depth=5, random_state=1)
score = np.mean(cross_val_score(ada2,X,y,scoring = 'accuracy',cv = crossvalidation,n_jobs = 1))
score

0.7301470588235295

you can see that the results are similar.This is just additional information that the gradient boosting model does outperform the baseline decision tree model.